# Ch. 5 Exercises
<ol>
<li>[Dynamic 1D Robot Motion Model](#Dynamic_1d_robot_motion)</li>
<li>[Dynamic 1D Robot Posterior](#Dynamic_1d_robot_posterior)</li>
<li>[Dynamic 1D Robot Correlation](#Dynamic_1d_robot_correlation)</li>
<li>[Dynamic Bicycle Model](#Dynamic_Bicycle_Model)</li>
<li>[Bicycle Sampling Function](#Bicycle_Sampling_Function)</li>
<li>[Bicycle Posterier Likelyhood](#Bicycle_Posterier_Likelyhood)</li>
</ol>

## 1. Dynamic 1D Robot Motion Model <a id="Dynamic_1d_robot_motion"></a>

Develop a mathematical motion model that computes the posterior over the pose $x^{\prime}$ and the velocity $\dot{x}^{\prime}$ from an initial pose $x$ and velocity $\dot{x}$ assuming the acceleration \ddot{x} is the sum of a command and zero-mean gaussian noise with variance $\sigma^{2}$. Are $x^{\prime}$ and $\dot{x}^{\prime}$ correlated in the posterior?


This is the same probem as in earlier chapters. Assuming constant acceleration and no noise during the movement:

\begin{equation}
\dot{x}^{\prime}  = x^{\prime} - x \\
\ddot{x} = \dot{x}^{\prime} - \dot{x} \\
\end{equation}


$x^{\prime}$ and $\dot{x}^{\prime}$ are correlated as one can be derived from the other if $\mu$ is known exactly.


## 2. Dynamic 1D Robot Motion Posterior <a id="Dynamic_1d_robot_posterior"></a>

The probability of an acceleration given the control is $p(\ddot{x} \lvert u_t) = Probability(\ddot{x} - u_t, \sigma^{2})$

The posterior can be calculated by

\begin{equation}
p(\mu^{\prime}\ \lvert\ \mu, u) = \left\{\begin{array}{cc}
0, & \dot{x}^{\prime} \ne x^{\prime} - x \\
Probability((\dot{x}^{\prime} - \dot{x}) - u_t, \sigma^{2}), & else \end{array}\right\}
\end{equation}

It seems like this could be made more general by adding another acceleration at the end of the move, similar to the extra rotation at the end of the odemetry model's move, so that that most states wouldn't have a zero probability.





## 3. Dynamic 1D Robot Motion Correlation <a id="Dynamic_1d_robot_correlation"></a>

After controlling the robot with random accelerations for $T$ time intervals, for some large value of $T$ the final location $x$ and final velocity $\dot{x}$ will not be correlated. 

## 4. Dynamic Bicycle Model <a id="Dynamic_Bicycle_Model"></a>

Create a dynamics model for a bicycle using three state variables $x,y$ of the center of front tire and $\theta$ yaw of frame relative to an external reference, and controls forward velocity $v$ and steering angle $\alpha$ after time $\Delta t$.

Assuming noiseless control and movement and bicyle length $L$, the bike will move in a circle whose center is located at the intersection of the lines that run perpedicualr through the center of the tires.

The radius of the circle is 
\begin{equation}
r_{front} = \left|\frac{L}{sin(\alpha)}\right| \\
r_{back} = \left|\frac{L}{tan(\alpha)}\right|
\end{equation}

The center of the circle is at 
\begin{equation}
x_{c} = x - \frac{L}{sin(\alpha)} * sin(\theta + \alpha) \\
y_{c} = y + \frac{L}{sin(\alpha)} * cos(\theta + \alpha)
\end{equation}

The control $v$ is assumed to be the speed of the front tire. The distance traveled around the circumference of the circle $d = v * \Delta t$. The angle traversed during that time is $\Delta\theta = \frac{d}{r}$. The new position of the front tire will then be 

\begin{equation}
x^{\prime} = x_{c} + \frac{L}{sin(\alpha)} * sin(\theta + \alpha + \Delta\theta) \\
y^{\prime} = y_{c} - \frac{L}{sin(\alpha)} * cos(\theta + \alpha + \Delta\theta)
\end{equation}

Combining the equations gives
\begin{equation}
x^{\prime} = x - \frac{L}{sin(\alpha)} * sin(\theta + \alpha) + \frac{L}{sin(\alpha)} * sin(\theta + \alpha + \Delta\theta) \\
y^{\prime} = y + \frac{L}{sin(\alpha)} * cos(\theta + \alpha) - \frac{L}{sin(\alpha)} * cos(\theta + \alpha + \Delta\theta)
\end{equation}

Which can be reduced to 
\begin{equation}
x^{\prime} = x + \frac{L}{sin(\alpha)} (-sin(\theta + \alpha) + sin(\theta + \alpha + \Delta\theta)) \\
y^{\prime} = y + \frac{L}{sin(\alpha)} ( cos(\theta + \alpha) - cos(\theta + \alpha + \Delta\theta))
\end{equation}



# Bicycle Model

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.animation as animation
%pylab inline

class Bicycle:
    
    def __init__(self, length=1, x=0, y=0, theta=0, epsilon=0.001):
        self.length = length
        self.x = x
        self.y = y
        self.theta = theta
        self.epsilon = epsilon
        
    def _get_center(self, alpha):
        if np.abs(alpha) < self.epsilon:
            return None
        
        r = self.length / np.sin(alpha)
        xc = self.x - r * np.sin(self.theta+alpha)
        yc = self.y + r * np.cos(self.theta+alpha)
        
        return (xc, yc, r)
    
    def _get_new_pos(self, alpha, v, delta_t=1):
        if np.abs(alpha) < self.epsilon:
            new_x = self.x + v * np.cos(self.theta) * delta_t
            new_y = self.y + v * np.sin(self.theta) * delta_t
            new_theta = self.theta
        else:
            xc, yc, r = self._get_center(alpha)
            
            delta_theta = (v * delta_t) / r
            
            new_x = xc + r * sin(self.theta + alpha + delta_theta)
            new_y = yc - r * cos(self.theta + alpha + delta_theta)
            new_theta = self.theta + delta_theta
        return (new_x, new_y, new_theta)


# Debug Plots

In [ ]:
def plot_bicycle(b, alpha, ax, plot_opts='k'):
    # front tire
    ax.plot([x - b.length / 3.0 * cos(b.theta+alpha), x + b.length / 3.0 * cos(b.theta+alpha)],
            [y - b.length / 3.0 * sin(b.theta+alpha), y + b.length / 3.0 * sin(b.theta+alpha)],
            plot_opts)
    
    # back tire
    bx = x - b.length * cos(b.theta)
    by = y - b.length * sin(b.theta)
    ax.plot([bx - b.length / 3.0 * cos(b.theta), bx + b.length / 3.0 * cos(b.theta)],
            [by - b.length / 3.0 * sin(b.theta), by + b.length / 3.0 * sin(b.theta)],
            plot_opts)

thetas = np.array([0, 45, 90, -30, 180]) * np.pi / 180.0
alphas = np.array([-45, -30, 0, 15]) * np.pi / 180.0
velocities = np.linspace(1, 4, 10)

x,y = 0,0
L = 1
b = Bicycle(L, x, y)
for theta in thetas:
    fig = plt.figure(figsize=(12,4))
    for i, alpha in enumerate(alphas):
        
        ax = fig.add_subplot(1,len(alphas),i+1, aspect='equal')
        b.theta = theta
        plot_bicycle(b, alpha, ax)
        
        if alpha != 0:
            xc, yc, r = b._get_center(alpha)
            ax.plot(xc,yc,'yo')

        for v in velocities:        
            xn, yn, tn = b._get_new_pos(alpha, v)
            
            bound = 5
            ax.set_xlim((-bound, bound))
            ax.set_ylim((-bound, bound))
            ax.plot(xn,yn,'go')
        


# Adding Noise
 
The textbook wants a general model assuming gaussian noise in the steering angle ( $\sigma^{2}_\alpha$ ) and commanded velocity ( $\sigma^{2}_v$ ).

Using a sammpling model it's trivial to add gausian noise to the steering angle and commanded velocity and run the equation again as if there is no noise.

If we are attempting to generate a posterior almost all of the states would have zero probablitiy because a final state that didn't have zero probablity would have to be tangent to a circle with the start point. The model can be improved to allow for a rotation before and after the move.

In [ ]:
# 
def prob_normal_distribution(a, b_sq):
    if b_sq == 0 and a == 0:
        return np.inf
    elif b_sq == 0 and a != 0:
        return 0.0
    return 1.0 / np.sqrt(2.0*np.pi*b_sq) * np.exp(-0.5 * a**2 / b_sq)

def prob_triangular_distribution(a, b_sq):
    if b_sq == 0 and a == 0:
        return np.inf
    elif b_sq == 0 and a != 0:
        return 0.0
    return max(0, 1.0 / np.sqrt(6.0 * b_sq) - np.abs(a) / (6*b_sq))

def sample_normal_distribution(b_sq):
    if b_sq == 0:
        return 0.0
    b = np.sqrt(b_sq)
    return 0.5 * np.random.uniform(-b, b, (12)).sum()

def sample_triangular_distribution(b_sq):
    if b_sq == 0:
        return 0.0
    b = np.sqrt(b_sq)
    return sqrt(6.0)*0.5 * (np.random.uniform(-b, b) + np.random.uniform(-b, b))


In [ ]:
# Vefity the probibility functions are working
norm_d = []
tri_d = []
xs = np.linspace(-3,3,101)
b_sq = 0.5
for a in xs:
    norm_d.append(prob_normal_distribution(a, b_sq))
    tri_d.append(prob_triangular_distribution(a, b_sq))
    
fig = plt.figure(figsize=(16,8))
ax1 = fig.add_subplot(1,2,1)
ax1.plot(xs, norm_d)
ax1.set_title('Normal Distribution')
ax1.set_xlim((-3,3))
ax1.set_ylim((0,0.6))
ax2 = fig.add_subplot(1,2,2)
ax2.plot(xs, tri_d)
ax2.set_title('Triangular Distribution')
ax2.set_xlim((-3,3))
ax2.set_ylim((0,0.6))

# Vefity the sample functions are working
N = 50000
norm_s = []
tri_s = []
for i in xrange(N):
    norm_s.append(sample_normal_distribution(b_sq))
    tri_s.append(sample_triangular_distribution(b_sq))

bins = 40 
fig = plt.figure(figsize=(16,8))
ax1 = fig.add_subplot(1,2,1)
ax1.hist(norm_s, bins=bins, range=(-3,3))

ax1.set_title('Normal Distribution')
ax1.set_xlim((-3,3))
ax1.set_ylim((0,5000))
ax2 = fig.add_subplot(1,2,2)
ax2.hist(tri_s, bins=bins, range=(-3,3))
ax2.set_title('Triangular Distribution')
ax2.set_xlim((-3,3))
ax2.set_ylim((0,5000))
pass

## 5. Bicycle Sampling Function <a id="Bicycle_Sampling_Function"></a>

param[x] = [$\alpha$ $(degrees)$, $v$ $\frac{cm}{sec}$]<br>
l = 100 cm,<br>
d = 80 cm,<br>
$\delta$t = 1 sec,<br>
$\left|\alpha\right|$ < 80 degrees,<br>
v 0 to 100 $\frac{cm}{sec}$,<br>
$\sigma^{2}_{\alpha}$ = 25 degrees$^{2}$,<br>
$\sigma^{2}_{v}$  = 50 $\frac{cm}{sec}^{2}$,<br>



In [ ]:
def sample(bicycle, alpha, v, sigma_sq_alpha, sigma_sq_v, delta_t, samples):
    new_positions = []
    for i in xrange(samples):
        new_alpha = alpha + sample_triangular_distribution(sigma_sq_alpha)
        new_v     = v     + sample_triangular_distribution(sigma_sq_v)
        
        # limit alpha to +/- 80 degrees
        new_alpha = max(new_alpha, np.deg2rad(-80))
        new_alpha = min(new_alpha, np.deg2rad(80))
        
        new_positions.append(bicycle._get_new_pos(new_alpha, new_v, delta_t))
    return new_positions

params = [( 25, 20),
          (-25, 20),
          ( 25, 90),
          ( 80, 10),
          ( 85, 90)]

params = [(p[0] * np.pi / 180.0, p[1]) for p in params]

x, y = 0, 0
L = 100
theta = 0
sigma_sq_alpha = 25 * np.pi / 180.0
sigma_sq_v = 50
b = Bicycle(L, x, y, theta)

new_positions = []
for alpha, v in params:
    sampled_positions = sample(b, alpha, v, sigma_sq_alpha, sigma_sq_v, delta_t=1.0, samples=500)
    new_positions.append(sampled_positions)
    
new_positions = np.array(new_positions)

# Plot the resutls
for run_params, results in zip(params, new_positions):
    fig = plt.figure(figsize=(10,4))
    ax = fig.add_subplot(1,1,1, aspect='equal')

    plot_bicycle(b, run_params[0], ax)

    ax.plot(results[:,0], results[:,1], 'go')


    

## 6. Bicycle Posterior Likelyhood <a id="Bicycle_Posterier_Likelyhood"></a>

Given inital params $x$, $y$, $\theta$ and final params $x'$, $y'$ but not $\theta'$, find the most probably values for $\alpha$, $v$, and $\theta'$.

Expanding the equations from earlier
\begin{equation}
x^{\prime} = x + \frac{L}{sin(\alpha)} (-sin(\theta + \alpha) + sin(\theta + \alpha + \Delta\theta)) \\
y^{\prime} = y + \frac{L}{sin(\alpha)} ( cos(\theta + \alpha) - cos(\theta + \alpha + \Delta\theta)) \\
\end{equation}

with $\theta^{\prime} = \theta + \Delta\theta = \theta  + \frac{v * \Delta t}{\left|\frac{L}{sin(\alpha)}\right|}$ gives
\begin{equation}
x^{\prime} = x + \frac{L}{sin(\alpha)} (-sin(\theta + \alpha) + sin(\theta^{\prime} + \alpha)) \\
y^{\prime} = y + \frac{L}{sin(\alpha)} ( cos(\theta + \alpha) - cos(\theta^{\prime} + \alpha)) \\
\end{equation}

Expanding the trig functions 
\begin{equation}
x^{\prime} = x + \frac{L}{sin(\alpha)} (-sin(\theta)*cos(\alpha) - cos(\theta)*sin(\alpha) + sin(\theta^{\prime})*cos(\alpha) + cos(\theta^{\prime})*sin(\alpha)) \\
y^{\prime} = y + \frac{L}{sin(\alpha)} (cos(\theta)*cos(\alpha) - sin(\theta)*sin(\alpha) - cos(\theta^{\prime})*cos(\alpha) + sin(\theta^{\prime})*sin(\alpha)) \\
\end{equation}

Multiplying through by 1/sin($\alpha$) and moving x and y to the left
\begin{equation}
x^{\prime} - x = L *((sin(\theta^{\prime}) -sin(\theta))*cot(\alpha) - cos(\theta) + cos(\theta^{\prime})) \\
y^{\prime} - y = L* ((cos(\theta)- cos(\theta^{\prime}))*cot(\alpha) - sin(\theta) + sin(\theta^{\prime})) \\
\end{equation}

Get alpha by itself
\begin{equation}
x^{\prime} - x + L*(cos(\theta) - cos(\theta^{\prime})) = L*((sin(\theta^{\prime}) - sin(\theta))*cot(\alpha)) \\
y^{\prime} - y + L*(sin(\theta) - sin(\theta^{\prime})) = L*((cos(\theta)- cos(\theta^{\prime}))*cot(\alpha)) \\
\end{equation}
\begin{equation}
\frac{\frac{\Delta x}{L} + (cos(\theta) - cos(\theta^{\prime}))}{sin(\theta^{\prime}) - sin(\theta)} = cot(\alpha) \\
\frac{\frac{\Delta y}{L} + (sin(\theta) - sin(\theta^{\prime}))}{cos(\theta)- cos(\theta^{\prime})} = cot(\alpha) \\
\end{equation}

Combine Equations
\begin{equation}
\frac{\frac{\Delta x}{L} + (cos(\theta) - cos(\theta^{\prime}))}{sin(\theta^{\prime}) - sin(\theta)} =
\frac{\frac{\Delta y}{L} + (sin(\theta) - sin(\theta^{\prime}))}{cos(\theta)- cos(\theta^{\prime})} \\
\end{equation}

Reduce
\begin{equation}
(\frac{\Delta x}{L} + cos(\theta) - cos(\theta^{\prime})) * (cos(\theta)- cos(\theta^{\prime})) = 
(\frac{\Delta y}{L} + sin(\theta) - sin(\theta^{\prime})) * (sin(\theta^{\prime}) - sin(\theta)) \\
\end{equation}

\begin{equation}
(\frac{\Delta x}{L} - 2*sin(\frac{\theta + \theta^{\prime}}{2})sin(\frac{\theta - \theta^{\prime}}{2}))) * - 2*sin(\frac{\theta + \theta^{\prime}}{2})sin(\frac{\theta - \theta^{\prime}}{2})) = 
(\frac{\Delta y}{L} + 2*sin(\frac{\theta - \theta^{\prime}}{2})cos(\frac{\theta + \theta^{\prime}}{2})) * 2*sin(\frac{\theta^{\prime} - \theta}{2})cos(\frac{\theta^{\prime} +\theta}{2})) \\
\end{equation}

Use $sin(-\theta) = -sin(\theta)$
\begin{equation}
(\frac{\Delta x}{L} - 2*sin(\frac{\theta + \theta^{\prime}}{2})sin(\frac{\theta - \theta^{\prime}}{2}))) * - 2*sin(\frac{\theta + \theta^{\prime}}{2})sin(\frac{\theta - \theta^{\prime}}{2})) = 
(\frac{\Delta y}{L} + 2*sin(\frac{\theta - \theta^{\prime}}{2})cos(\frac{\theta + \theta^{\prime}}{2})) *  - 2*sin(\frac{\theta - \theta^{\prime}}{2})cos(\frac{\theta^{\prime} +\theta}{2})) \\
\end{equation}

\begin{equation}
(\frac{\Delta x}{L} - 2*sin(\frac{\theta + \theta^{\prime}}{2})sin(\frac{\theta - \theta^{\prime}}{2}))) * sin(\frac{\theta + \theta^{\prime}}{2})) = 
(\frac{\Delta y}{L} + 2*sin(\frac{\theta - \theta^{\prime}}{2})cos(\frac{\theta + \theta^{\prime}}{2})) *cos(\frac{\theta^{\prime} +\theta}{2})) \\
\end{equation}

\begin{equation}
\frac{\Delta x}{L} * sin(\frac{\theta + \theta^{\prime}}{2}) - 2*sin(\frac{\theta + \theta^{\prime}}{2})^{2}sin(\frac{\theta - \theta^{\prime}}{2})) = 
\frac{\Delta y}{L} *cos(\frac{\theta^{\prime} +\theta}{2}) + 2*sin(\frac{\theta - \theta^{\prime}}{2})cos(\frac{\theta + \theta^{\prime}}{2})^{2} \\
\end{equation}

\begin{equation}
\frac{\Delta x}{L} * sin(\frac{\theta + \theta^{\prime}}{2}) - \frac{\Delta y}{L} *cos(\frac{\theta^{\prime} +\theta}{2}) = 
2*sin(\frac{\theta - \theta^{\prime}}{2})cos(\frac{\theta + \theta^{\prime}}{2})^{2} + 2*sin(\frac{\theta + \theta^{\prime}}{2})^{2}sin(\frac{\theta - \theta^{\prime}}{2}))\\
\end{equation}

\begin{equation}
\frac{\Delta x}{L} * sin(\frac{\theta + \theta^{\prime}}{2}) - \frac{\Delta y}{L} *cos(\frac{\theta^{\prime} +\theta}{2}) = 
2*sin(\frac{\theta - \theta^{\prime}}{2}) * (sin(\frac{\theta + \theta^{\prime}}{2})^{2} + cos(\frac{\theta + \theta^{\prime}}{2})^{2})\\
\end{equation}

\begin{equation}
\frac{\Delta x}{L} * sin(\frac{\theta + \theta^{\prime}}{2}) - \frac{\Delta y}{L} *cos(\frac{\theta^{\prime} +\theta}{2}) = 
2*sin(\frac{\theta - \theta^{\prime}}{2})\\
\end{equation}

\begin{equation}
\frac{\Delta x}{L}*sin(\frac{\theta}{2})cos(\frac{\theta^{\prime}}{2}) + 
\frac{\Delta x}{L}*cos(\frac{\theta}{2})sin(\frac{\theta^{\prime}}{2}) - 
\frac{\Delta y}{L}*cos(\frac{\theta}{2})cos(\frac{\theta^{\prime}}{2}) - 
\frac{\Delta y}{L}*sin(\frac{\theta}{2})sin(\frac{\theta^{\prime}}{2}) = 
2*sin(\frac{\theta}{2})cos(\frac{\theta^{\prime}}{2}) - 
2*cos(\frac{\theta}{2})sin(\frac{\theta^{\prime}}{2})\\
\end{equation}

\begin{equation}
(\frac{\Delta x}{L} - 2)*sin(\frac{\theta}{2})cos(\frac{\theta^{\prime}}{2}) + 
(\frac{\Delta x}{L} + 2)*cos(\frac{\theta}{2})sin(\frac{\theta^{\prime}}{2}) - 
\frac{\Delta y}{L}*cos(\frac{\theta}{2})cos(\frac{\theta^{\prime}}{2}) - 
\frac{\Delta y}{L}*sin(\frac{\theta}{2})sin(\frac{\theta^{\prime}}{2}) = 0
\end{equation}

\begin{equation}
- ((\frac{\Delta x}{L} - 2)*sin(\frac{\theta}{2}) - \frac{\Delta y}{L}*cos(\frac{\theta}{2})) cos(\frac{\theta^{\prime}}{2}) = 
((\frac{\Delta x}{L} + 2)*cos(\frac{\theta}{2}) - \frac{\Delta y}{L}*sin(\frac{\theta}{2}))
sin(\frac{\theta^{\prime}}{2})
\end{equation}

\begin{equation}
\frac{-(\frac{\Delta x}{L} - 2)*sin(\frac{\theta}{2}) - \frac{\Delta y}{L}*cos(\frac{\theta}{2})}
     {(\frac{\Delta x}{L} + 2)*cos(\frac{\theta}{2}) - \frac{\Delta y}{L}*sin(\frac{\theta}{2})} = 
tan(\frac{\theta^{\prime}}{2})
\end{equation}

\begin{equation}
\theta^{\prime} = 2 * arctan \left(
\frac{-(\frac{\Delta x}{L} - 2)*sin(\frac{\theta}{2}) - \frac{\Delta y}{L}*cos(\frac{\theta}{2})}
     {(\frac{\Delta x}{L} + 2)*cos(\frac{\theta}{2}) - \frac{\Delta y}{L}*sin(\frac{\theta}{2})}
     \right)
\end{equation}
